In [ ]:
# !kaggle datasets download jrobischon/wikipedia-movie-plots
# !kaggle datasets download cryptexcode/mpst-movie-plot-synopses-with-tags

Dataset URL: https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots
License(s): CC-BY-SA-4.0
Dataset URL: https://www.kaggle.com/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags
License(s): CC-BY-SA-4.0


In [ ]:
# import zipfile
# import os

# zip_files = ["mpst-movie-plot-synopses-with-tags.zip", "wikipedia-movie-plots.zip"]

# for zip_file in zip_files:
#     zip_path = os.path.join(os.getcwd(), zip_file)
    
#     if not os.path.exists(zip_path):
#         print(f"{zip_file} not found!")
#         continue
    
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(os.getcwd()) 
#         print(f"{zip_file} unziped!")

mpst-movie-plot-synopses-with-tags.zip unziped!
wikipedia-movie-plots.zip unziped!


In [ ]:
import pandas as pd
import os

df1 = pd.read_csv("mpst_full_data.csv") 
df2 = pd.read_csv("wiki_movie_plots_deduped.csv") 

df1 = df1.rename(columns={'title': 'title', 'plot_synopsis': 'plot'}) 
df2 = df2.rename(columns={'Title': 'title', 'Plot': 'plot'}) 

df1 = df1[['title', 'plot']]
df2 = df2[['title', 'plot']] 

combined_df = pd.concat([df1, df2], ignore_index=True)

final_df = combined_df.drop_duplicates(subset='title', keep='first')

combined_df.shape

(49714, 2)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(final_df['plot'].tolist(), convert_to_tensor=False)

query = "Crime drama about drug cartels"
query_embedding = model.encode(query, convert_to_tensor=False).reshape(1, -1)

cos_scores = cosine_similarity(query_embedding, embeddings)
top_results = np.argsort(-cos_scores.flatten())[:5]

for idx in top_results:
    print(f"Movie: {final_df.iloc[idx]['title']}, Similarity: {cos_scores[0, idx]:.4f}")

Movie: This Rebel Breed, Similarity: 0.6487
Movie: Buying Time, Similarity: 0.6411
Movie: Traffic, Similarity: 0.6043
Movie: Port of New York, Similarity: 0.5899
Movie: Johnny Stool Pigeon, Similarity: 0.5810


In [ ]:
def precision_at_k(relevant, retrieved, k):
    retrieved_at_k = retrieved[:k]
    relevant_set = set(relevant)
    retrieved_set = set(retrieved_at_k)
    return len(relevant_set & retrieved_set) / k if k > 0 else 0

def recall_at_k(relevant, retrieved, k):
    retrieved_at_k = retrieved[:k]
    relevant_set = set(relevant)
    retrieved_set = set(retrieved_at_k)
    return len(relevant_set & retrieved_set) / len(relevant_set) if len(relevant_set) > 0 else 0

def average_precision(relevant, retrieved):
    relevant_set = set(relevant)
    ap = 0.0
    num_relevant = 0
    
    for i, doc_id in enumerate(retrieved):
        if doc_id in relevant_set:
            num_relevant += 1
            precision = num_relevant / (i + 1)
            ap += precision
    
    return ap / len(relevant_set) if len(relevant_set) > 0 else 0

def mean_average_precision(queries_results):
    return np.mean([ap for _, ap in queries_results])

def normalized_dcg(relevant, retrieved, k, relevances=None):
    if not relevant:
        return 0.0

    # Binary relevance if not provided
    if relevances is None:
        relevances = {doc_id: 1 for doc_id in relevant}
    
    # DCG
    dcg = 0.0
    for i, doc_id in enumerate(retrieved[:k]):
        rel = relevances.get(doc_id, 0)
        if i == 0:
            dcg += rel
        else:
            dcg += rel / np.log2(i + 1)
    
    # Ideal DCG
    ideal_relevances = sorted(relevances.values(), reverse=True)[:k]
    idcg = 0.0
    for i, rel in enumerate(ideal_relevances):
        if i == 0:
            idcg += rel
        else:
            idcg += rel / np.log2(i + 1)
    
    return dcg / idcg if idcg > 0 else 0